# 1.1 LangChain and LangGraph (feat. OpenAI ChatGPT)

- LangChain과 LangGraph를 활용해서 OpenAI의 ChatGPT를 호출 (`llm.invoke()`). 
- LangGraph의 개념(state, node, edge)과 주요 기능을 이해.

## 환경설정

- `LangChain/LangGraph` 활용을 위해 필요한 패키지들을 설치합니다.
- `.env` 파일을 생성하여 환경변수를 설정합니다.

In [1]:
%pip install -q python-dotenv langchain-openai langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## LangChain

- `LangChain`은 **Runable**을 만들고 `invoke()`함수로 실행하는 구조

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

query = "홍익대학교에 대해서 50자 내로 설명해줘."
response = llm.invoke(query)

response

- 여러개의 **Runable**을 자유자제로 이어서 하나의 Chain Runable을 생성

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    template="""당신은 유능한 어시스턴트입니다.

    사용자의 질문에 대해 100자 내로 답변해주세요.
    
    ## 질문: {query}
    """)

query = "홍익대학교에 대해서 50자 내로 설명해줘."
chain = prompt | llm
response = chain.invoke({"query": query})

response

## LangGraph
- `LangGraph`는 state, node, edge로 구성된 graph(workflow)를 생성하고, 이를 실행하는 구조

### state
- `state`는 LangGraph 에이전트의 state를 나타내는 데이터 구조입니다.
- `state`는 `TypedDict`를 사용하여 정의되며, 이는 Python의 타입 힌팅을 통해 구조를 명확히 합니다.
    - 지금 예제에서는 간단하게 `messages`라는 필드만 있습니다.
    - 필요에 따라 다양한 값들을 활용할 수 있습니다.
- `state`는 그래프 내에서 유지되는 메모리로 사용되며, 각 노드에서 state를 업데이트하거나 참조할 수 있습니다.
- `state`는 LangGraph의 노드 간에 전달되며, 에이전트의 state 전이를 관리합니다.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class TutorialState(TypedDict):
    messages: list[Annotated[AnyMessage,add_messages]]


- 위에 선언한 `TutorialState`를 활용하여 `StateGraph`를 생성합니다.

In [ ]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(TutorialState)

### node
- `graph`에 추가할 `node`를 생성합니다
-  `node`는 LangGraph에서 실행되는 개별적인 작업 단위를 의미합니다. 
    - 각 노드는 특정 기능을 수행하는 독립적인 컴포넌트로, 예를 들어 텍스트 생성, 데이터 처리, 또는 의사 결정과 같은 작업을 담당할 수 있습니다.
    - `node`는 기본적으로 함수(function)로 정의되고, 에이전트(agent)로 활용할 수도 있습니다

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def generate(state: TutorialState) -> TutorialState:
    """
    `generate` 노드는 사용자의 질문을 받아서 응답을 생성하는 노드입니다.
    """
    messages = state['messages']
    ai_message = llm.invoke(messages)
    return {'messages': [ai_message]}

- graph에 생성한 node를 추가해줍니다.

In [ ]:
graph_builder.add_node('generate', generate)

### edge
- `node`를 생성한 후에 `edge`로 연결합니다
- `edge`는 노드들 사이의 연결을 나타내며, 데이터와 제어 흐름의 경로를 정의합니다. 
    - 엣지를 통해 한 노드의 출력이 다음 노드의 입력으로 전달되어, 전체적인 워크플로우가 형성됩니다.
    - `node`와 `edge`의 조합은 방향성 그래프(Directed Graph)를 형성하며, 이를 통해 복잡한 AI 에이전트의 행동 흐름을 구조화할 수 있습니다
- 모든 그래프는 `START(시작)`와 `END(종료)`가 있습니다
    - `END`를 명확하게 선언하지 않는 경우도 종종 있지만, 가독성을 위해 작성해주는 것을 권장합니다

In [ ]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

- `node`를 생성하고 `edge`로 연결한 후에 `compile` 메서드를 호출하여 `Graph`를 생성합니다

In [ ]:
tutorial_graph = graph_builder.compile()

- `compile` 후에는 그래프를 시각화하여 확인할 수 있습니다
- 의도한대로 그래프가 생성됐는지 확인하는 습관을 기르는 것이 좋습니다

In [ ]:
from IPython.display import display, Image

display(Image(tutorial_graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

query = "경상국립대학교에 대해서 50자 내로 설명해줘."
initial_state = {'messages': [HumanMessage(query)]}

response = tutorial_graph.invoke(initial_state)
response
# print(response['messages'][-1].content)